# Active Learning

In [ ]:
import pandas as pd
import numpy as np
import uuid
import random

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, matthews_corrcoef, confusion_matrix, classification_report

## Generate Data

In case this needs to be said, if you're using this approach to solve a problem prominent to you, don't bother generating a dataset, use the one you're working with. I'm only generating a dataset for simplicity, reproducibility and to outline a use case on how such a problem could be solved using active learning.  

We are going to generate fake vital statistics associated to humans. The function we're building will randomly generate fake height, weight and age data. 

In [ ]:
def generate_data(n = 1000):
    '''
    This function will simulate the generation of gender data. It will randomly create
    columns associated to the height (cm), weight, age and gender. The gender column
    will not be completely filled out, it will have a few sample rows labelled to a gender.
    
    params:
        n (Integer) : The number of rows you want to synthesize
        
    returns:
        A dataframe with the columns of uuid, height, weight, age and gender.
            - uuid (UUID4) : A unique identifier to the user
            - height (Integer) : The height of the user in cm
            - weight (Integer) : The weight of the user in pounds (lbs)
            - age (Integer) : The age of the user
            - gender (String) : The gender associated to the user if known
        
    example:
        gender_df = generate_data(n = 1000)
    '''
    # we have more np.nan than Male or Female so that we can skew majority of the
    # data to be missing
    genders = ['Male', 'Female', np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
    height_range = (50, 200)
    weight_range = (30, 250)
    age_range = (3, 70)
    
    d = pd.DataFrame(
        {
            'uuid' : [uuid.uuid4() for _ in range(n)],
            'height' : [random.randint(height_range[0], height_range[1]) for _ in range(n)],
            'weight' : [random.randint(weight_range[0], weight_range[1]) for _ in range(n)],
            'age' : [random.randint(age_range[0], age_range[1]) for _ in range(n)],
            'gender' : [random.choice(genders) for _ in range(n)]
        }
    ).drop_duplicates()
    d = d.set_index('uuid')
    return d

In [ ]:
gender_df = generate_data(n = 1000)
gender_df.shape

In [ ]:
gender_df.head()

Of course, since we randomly generated data, we will have weird labelled values, like a 39 year old man who weighs 214 pounds and is 78 centimeters tall. 

In [ ]:
gender_df.describe()

## Annotate Data

Here people would usually annotate their datasets associated to the labels, 

In [ ]:
gender_df['gender'].value_counts()

In [ ]:
train_df = gender_df[~gender_df['gender'].isna()].copy()
pred_df = gender_df[gender_df['gender'].isna()].copy()
print(train_df.shape, pred_df.shape)

## Train Model

In [ ]:
# feature, target breakdown
ft_cols = train_df.drop(columns = ['gender']).columns.tolist()
target_col = 'gender'

# train test split
x = train_df[ft_cols].values
y = train_df[target_col].values

x_train, x_test, y_train, y_test = train_test_split(
    x, 
    y,
    test_size = 0.3
)

In [ ]:
# GBC classifier
clf = GradientBoostingClassifier()

# train the model
clf.fit(x_train, y_train)

## Model Accuracy

In [ ]:
def clf_eval(clf, x_test, y_test):
    '''
    This function will evaluate a sk-learn multi-class classification model based on its
    x_test and y_test values
    
    params:
        clf (Model) : The model you wish to evaluate the performance of
        x_test (Array) : Result of the train test split
        y_test (Array) : Result of the train test split
    
    returns:
        This function will return the following evaluation metrics:
            - Accuracy Score
            - Matthews Correlation Coefficient
            - Classification Report
            - Confusion Matrix
    
    example:
        clf_eval(
            clf,
            x_test,
            y_test
        )
    '''
    y_pred = clf.predict(x_test)
    y_true = y_test
    
    y_pred = clf.predict(x_test)
    test_acc = accuracy_score(y_test, y_pred)
    print("Testing Accuracy : ", test_acc)
    
    print("MCC Score : ", matthews_corrcoef(y_true, y_pred))
    
    print("Classification Report : ")
    print(classification_report(y_test, clf.predict(x_test)))
    
    
clf_eval(
    clf,
    x_test,
    y_test
)

## Predict

In [ ]:
pred_df['pred_proba'] = pred_df[ft_cols].apply(lambda x : dict(
    zip(clf.classes_, clf.predict_proba(x.values[None])[0])
), axis = 1)
pred_df['pred'] = pred_df[ft_cols].apply(lambda x : clf.predict(x.values[None])[0], axis = 1)

## Get Annotated & To Annoate Predictions

Predictions with low predicted probabilities will be annotated manually by the user, while predictions which have high predicted probability will be assumed to be a good prediction generated by the model. This will allow us to increase our labelled data and retrain a new model based on the best performing results of the previous model.

In [ ]:
# low predictions with threshold <= 0.6
low_th= 0.6

# high predictions with threshold >= 0.9
high_th = 0.9

def check_preds(pred_dct, low_th = low_th, high_th = high_th):
    '''
    This function will check the dictionary associated to the prediction probabilities
    generated by the model. It will return either `annotate`, `annotated` or `neither`
    as a result.
    
    params:
        pred_dct (Dictionary) : The keys are the classes, values are the predicted proba
        low_th (Integer) : The low prediction proba threshold
        high_th (Integer): The high prediction proba threshold
        
    returns:
        This function will return the following:
            `annotate` : if the maximum value in the pred_dct is less than or equal to
                         the low_th
            `annotated` : if the maximum value in the pred_dct is greater than or equal to
                          the high_th
            'neither' : If it does not fall in the other two ranges
            
    example:
        pred_df['annotate_decision'] = pred_df['pred_proba'].apply(check_preds)
    '''
    
    max_val = max(list(pred_dct.values()))
    if max_val <= low_th:
        return 'annotate'
    elif max_val >= high_th:
        return 'annotated'
    else:
        return 'neither'

In [ ]:
pred_df['annotate_decision'] = pred_df['pred_proba'].apply(check_preds)

In [ ]:
pred_df['annotate_decision'].value_counts()

In [ ]:
annotated_df = pred_df[pred_df['annotate_decision'] == 'annotated'].copy()
annotate_df = pred_df[pred_df['annotate_decision'] == 'annotate'].copy()

I'm just going to randomly assign a gender out of simplicity (since this is randomly generated data), but in reality you should manually annotate this.

In [ ]:
gender_choices = ['Male', 'Female', np.nan, np.nan, np.nan]
annotate_df['gender'] = [random.choice(gender_choices) for _ in range(annotate_df.shape[0])]
annotate_df = annotate_df[~annotate_df['gender'].isna()].copy()

## Update Labelled Data

In [ ]:
annotated_df['gender'] = annotated_df['pred']

In [ ]:
annotated_df = pd.concat([annotated_df, annotate_df, train_df])

In [ ]:
annotated_df.shape

## Retrain Model

In [ ]:
# feature, target breakdown
ft_cols = ['height', 'weight', 'age']
target_col = 'gender'

# train test split
x = annotated_df[ft_cols].values
y = annotated_df[target_col].values

x_train, x_test, y_train, y_test = train_test_split(
    x, 
    y,
    test_size = 0.3
)

In [ ]:
# GBC classifier
clf = GradientBoostingClassifier()

# train the model
clf.fit(x_train, y_train)

## Model Accuracy

In [ ]:
clf_eval(
    clf,
    x_test,
    y_test
)

## Concluding Remarks

---